In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline

In [2]:
pd.options.display.max_colwidth=100
np.set_printoptions(linewidth=140,edgeitems=10)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
feedbacks=[]
with open('train_feedback.txt') as f:
    for line in f:
        feedback = eval(line)
        feedbacks.append(feedback)

In [4]:
test_feedbacks=[]
with open('test_feedback.txt') as f:
    for line in f:
        test_feedback = eval(line)
        test_feedbacks.append(test_feedback)

In [5]:
feedbacks = pd.DataFrame(feedbacks)

In [6]:
descriptions=[]

with open("movie_descriptions.txt", encoding="utf-8") as f:
    for line in f:
        description = eval(line)
        descriptions.append(description)

In [7]:
descriptions = pd.DataFrame(descriptions)

In [8]:
Z=pd.merge(feedbacks,descriptions,on='MovieId')

In [9]:
for index, row in Z.iterrows():
    Z.set_value(index, 'text', Z['text'][index].lower()
                .replace('.', ' ')
                .replace(',', ' ')
                .replace(';', ' ')
                .replace(':', ' ')
                .replace('!', ' ')
                .replace('?', ' ')
                .replace('\r', ' ')
                .replace('\n', ' ')
               )
    Z.set_value(index, 'summary', Z['summary'][index].lower()
                .replace('.', ' ')
                .replace(',', ' ')
                .replace(';', ' ')
                .replace(':', ' ')
                .replace('!', ' ')
                .replace('?', ' ')
                .replace('\r', ' ')
                .replace('\n', ' ')
               )

In [10]:
test_feedbacks = pd.DataFrame(test_feedbacks)

In [11]:
y_train = Z['score']

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [93]:
from sklearn.feature_extraction.text import  CountVectorizer


In [94]:
def extract_features_cv(train_texts, test_texts, ngrams_count, stp_wrds=None, mindf=2, maxdf=0.9):
    
    tfidf = CountVectorizer(ngram_range=(1, ngrams_count), stop_words=stp_wrds, min_df=mindf, max_df=maxdf)
    train_features = tfidf.fit_transform(train_texts)
    test_features = tfidf.transform(test_texts)
    return train_features, test_features

In [95]:
tit_train_features_cv, tit_test_features_cv = extract_features_cv(Z["text"].values,
                                                                  test_feedbacks["text"].values, 2, 'english', 5, 0.9)

In [97]:
body_train_features_cv, body_test_features_cv = extract_features_cv(Z["summary"].values,
                                                                    test_feedbacks["summary"].values, 2, 'english', 5, 0.9)

In [27]:
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
text_train_f = vectorizer.fit_transform(Z["text"] )
text_test_f = vectorizer.transform(test_feedbacks["text"])

In [28]:
summary_vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
summary_train_f = summary_vectorizer.fit_transform(Z["summary"])
summary_test_f = summary_vectorizer.transform(test_feedbacks["summary"])

In [29]:
test_feedbacks.shape[0]

66953

In [98]:
train_f = sp.hstack((text_train_f, summary_train_f,body_train_features_cv,tit_train_features_cv), format='csr')

In [99]:
test_f = sp.hstack((text_test_f, summary_test_f,body_test_features_cv,tit_test_features_cv), format='csr')

In [100]:
test_f.shape

(66953, 3325154)

In [101]:
train_f.shape

(66953, 3325154)

In [102]:
clf = LogisticRegression(C=1.0)

In [103]:
clf.fit(train_f, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [104]:
Y_pred = clf.predict(test_f)

In [105]:
Y = pd.DataFrame(Y_pred)

In [106]:
content = pd.concat([test_feedbacks.id, Y],axis = 1)
content.shape

(66953, 2)

In [73]:
content.head()

,id,0
0,32551,5
1,80268,5
2,78875,5
3,47745,2
4,105443,5


In [107]:
content.columns=['ReviewId', 'Score']

In [108]:
content.to_csv('sol.csv',index=False)